In [1]:
import yfinance as yf
import pandas as pd
import math as math
from yahoofinancials import YahooFinancials
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

yeardata = ''

def collector(num, df):
    array = []
    if df is None:
        return 'STOP'
    else:
        for y in range(len(df.columns)):
            y = y+1
            item = df.loc[num, y]
            if item is None:
                return 'STOP'
            if math.isnan(item) == True:
                return 'STOP'
            array.append(item)

        return array

def dataframe(symbol, indexname):
    if (indexname == 'financials'):
        info = symbol.financials
    elif (indexname == 'cashflow'):
        info = symbol.cashflow
    elif (indexname == 'balance_sheet'):
        info = symbol.balance_sheet
    df = pd.DataFrame(data=info)
    df = df.reset_index()
    recent = str(df.columns[1])
    arg = '2022' in recent
    if arg == True:
        global yeardata
        yeardata = '2022'
        
    if len(df.columns) == 5:
        df.columns = ['Stats', 4, 3, 2, 1]
        df = df.drop('Stats', axis=1)
        df.columns = [4, 3, 2, 1]
        return df
    elif len(df.columns) == 4:
        df.columns = ['Stats', 3, 2, 1]
        df = df.drop('Stats', axis=1)
        df.columns = [3, 2, 1]
        return df
    elif len(df.columns) == 3:
        df.columns = ['Stats', 2, 1]
        df = df.drop('Stats', axis=1)
        df.columns = [2, 1]
        return df
    elif len(df.columns) == 2:
        df.columns = ['Stats', 1]
        df = df.drop('Stats', axis=1)
        df.columns = [1]
        return df
    

def sharenumber(stringsymbol):
    driver = webdriver.Chrome('/Users/sumandeepchahal/Desktop/StockProgramming/chromedriver')
    link = 'https://ycharts.com/companies/' + stringsymbol + '/shares_outstanding'
    driver.get(link)
    
    try:
        sharedv = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.XPATH, "//div[@class='row']/div[@class='col-6']/table[@class='table']/tbody/tr/td")))
        sharesoutstanding = []
    
        for data in sharedv:
            data = data.text
            sharesoutstanding.append(data)

        years = []
        years1 = []

        y21 = 0
        y20 = 0
        y19 = 0
        y18 = 0

        for x in range(len(sharesoutstanding)):    
            if "," in sharesoutstanding[x]:
                distorteddate = sharesoutstanding[x].split(", ")
                del distorteddate[0]
                sharesoutstanding[x] = str(distorteddate[0])
                if y21 == 0:
                    if "2021" in sharesoutstanding[x]:
                        years.append(x)
                        years1.append(sharesoutstanding[x])
                        y21 = 1
                elif y20 == 0:
                    if "2020" in sharesoutstanding[x]:
                        years.append(x)
                        years1.append(sharesoutstanding[x])
                        y20 = 1
                elif y19 == 0:
                    if "2019" in sharesoutstanding[x]:
                        years.append(x)
                        years1.append(sharesoutstanding[x])
                        y19 = 1
                elif y18 == 0:
                    if "2018" in sharesoutstanding[x]:
                        years.append(x)
                        years1.append(sharesoutstanding[x])
                        y18 = 1

        shareso = []

        for i in range(len(years)):
            index = years[i]+1
            sout = sharesoutstanding[index]
            if "T" in sout:
                sout = sout.split("T")
                sout = float(sout[0])*1000000000000
                sharesoutstanding[index] = sout
                shareso.append(sout)
            elif "B" in sout:
                sout = sout.split("B")
                sout = float(sout[0])*1000000000
                sharesoutstanding[index] = sout
                shareso.append(sout)
            elif "M" in sout:
                sout = sout.split("M")
                sout = float(sout[0])*1000000
                sharesoutstanding[index] = sout
                shareso.append(sout)

        years1 = list(reversed(years1))
        shareso = list(reversed(shareso))

        if len(shareso) == 4:
            return shareso
        elif len(shareso) > 0:
            compiledarray = []
            compiledarray.append(shareso)
            compiledarray.append(years1)
            return compiledarray
    
    except:
        return 'STOP'
    
    driver.quit()
    
def getvalues(symbol, stringsymbol):
    df = dataframe(symbol, 'financials')
    
    if df is not None:
        if len(df.index) > 15:  
            if collector(num=15, df=df) == 'STOP':
                revenue = 'NOTFOUND'
            else:
                revenue = collector(num=15, df=df)


            if collector(num=6, df=df) == 'STOP':
                grossprofit = 'NOTFOUND'
            else:
                grossprofit = collector(num=6, df=df)


            if collector(num=4, df=df) == 'STOP':
                netincome = 'NOTFOUND'
            else:
                netincome = collector(num=4, df=df)

            if revenue != 'NOTFOUND':
                netprofitmargin = []
                for y in range(len(revenue)):
                    if revenue[y] == 0:
                        netprofitmargin = 'NOTFOUND'
                        break
                    netprofitmitem = (netincome[y]/revenue[y]) * 100
                    netprofitmargin.append(netprofitmitem)
            else:
                netprofitmargin = 'NOTFOUND'
        else:
            revenue = 'NOTFOUND'
            grossprofit = 'NOTFOUND'
            netincome = 'NOTFOUND'
            netprofitmargin = 'NOTFOUND'
    else:
        revenue = 'NOTFOUND'
        grossprofit = 'NOTFOUND'
        netincome = 'NOTFOUND'
        netprofitmargin = 'NOTFOUND'
        
    del df
    
    df = dataframe(symbol, 'cashflow')
    
    if df is not None:
        if len(df.index) > 11:
            if collector(num=1, df=df) == 'STOP':
                changetoliab = 'NOTFOUND'
            else:
                changetoliab = collector(num=1, df=df)

            #CHECKLEN OF INDEX
            if collector(num=8, df=df) == 'STOP':
                changeincash = 'NOTFOUND'
            else:
                changeincash = collector(num=8, df=df)

            if collector(num=10, df=df) == 'STOP':
                totalcashfromoperations = 'NOTFOUND'
            else:
                totalcashfromoperations = collector(num=10, df=df)

        else:
            changetoliab = 'NOTFOUND'
            changeincash = 'NOTFOUND'
            totalcashfromoperations = 'NOTFOUND'
    else:
        changetoliab = 'NOTFOUND'
        changeincash = 'NOTFOUND'
        totalcashfromoperations = 'NOTFOUND'
    
    del df
    
    df = dataframe(symbol, 'balance_sheet')
    
    if df is not None:
        if len(df.index) > 11:
            if collector(num=10, df=df) == 'STOP':
                cash = 'NOTFOUND'
            else:
                cash = collector(num=10, df=df)

            if collector(num=3, df=df) == 'STOP':
                totalassets = 'NOTFOUND'
            else:
                totalassets = collector(num=3, df=df)

            if collector(num=0, df=df) == 'STOP':
                totalliabilities = 'NOTFOUND'
            else:
                totalliabilities = collector(num=0, df=df)

        else:
            cash = 'NOTFOUND'
            totalassets = 'NOTFOUND'
            totalliabilities = 'NOTFOUND'
    else:
        cash = 'NOTFOUND'
        totalassets = 'NOTFOUND'
        totalliabilities = 'NOTFOUND'
    
    del df
    
    dict = symbol.info
    
    arg = 'currentRatio' in dict
    if (arg == True):
        currentratio = dict['currentRatio']
    else:
        currentratio = 'NOTFOUND'
    
    arg = 'quickRatio' in dict
    if (arg == True):
        quickratio = dict['quickRatio']
    else:
        quickratio = 'NOTFOUND'
    
    arg = 'forwardEps' in dict
    if (arg == True):
        forwardeps = dict['forwardEps']
    else:
        forwardeps = 'NOTFOUND'
    
    arg = 'trailingEps' in dict
    if (arg == True):
        trailingeps = dict['trailingEps']
    else:
        trailingeps = 'NOTFOUND'
    
    arg = 'trailingPE' in dict
    if (arg == True):
        trailingpe = dict['trailingPE']
    else:
        trailingpe = 'NOTFOUND'
        
    arg = 'forwardPE' in dict
    if (arg == True):
        forwardpe = dict['forwardPE']
    else:
        forwardpe = 'NOTFOUND'
    
    
    pricedata = yf.download(stringsymbol, '2018-01-01', '2022-01-01')
    df = pd.DataFrame(data=pricedata)
    df = df.reset_index()
    df.columns = [0, 1, 2, 3, 4, 5, 6]
    
    pricearray1 = []
    pricearray2 = []
    pricearray3 = []
    pricearray4 = []
    
    for u in range(len(df.index)):
        year = df.loc[u, 0]
        year = year.date()
        year = str(year)
        if '2018' in year:
            price = df.loc[u, 4]
            if math.isnan(price) == False:
                pricearray1.append(price)
        elif '2019' in year:
            price = df.loc[u, 4]
            if math.isnan(price) == False:
                pricearray2.append(price)
        elif '2020' in year:
            price = df.loc[u, 4]
            if math.isnan(price) == False:
                pricearray3.append(price)
        elif '2021' in year:
            price = df.loc[u, 4]
            if math.isnan(price) == False:
                pricearray4.append(price)
    if len(pricearray1) > 0:
        price1 = pricearray1[-1]
    else:
        price1 = 0
    
    if len(pricearray2) > 0:
        price2 = pricearray2[-1]
    else:
        price2 = 0
    
    if len(pricearray3) > 0:
        price3 = pricearray3[-1]
    else:
        price3 = 0
    
    if len(pricearray4) > 0:
        price4 = pricearray4[-1]
    else:
        price4 = 0
    
    prices = [price1, price2, price3, price4]
    
    del df
    
    #selenium webscraping of Ycharts
    sharesoutstanding = sharenumber(stringsymbol)
    
    if sharesoutstanding != 'STOP' and sharesoutstanding is not None:
        marketcaps = []
        marketcaps1 = []

        if len(sharesoutstanding) == 4:
            for i in range(len(sharesoutstanding)):
                mcap = sharesoutstanding[i] * prices[i]
                marketcaps.append(mcap)

        elif len(sharesoutstanding) > 0:
            years1 = sharesoutstanding[1]
            for i in range(len(sharesoutstanding[1])):
                if sharesoutstanding[1][i] == "2018":
                    mcap = prices[0] * sharesoutstanding[0][i]
                    marketcaps1.append(mcap)
                elif sharesoutstanding[1][i] == "2019":
                    mcap = prices[1] * sharesoutstanding[0][i]
                    marketcaps1.append(mcap)
                elif sharesoutstanding[1][i] == "2020":
                    mcap = prices[2] * sharesoutstanding[0][i]
                    marketcaps1.append(mcap)
                elif sharesoutstanding[1][i] == "2021":
                    mcap = prices[3] * sharesoutstanding[0][i]
                    marketcaps1.append(mcap)

            marketcaps.append(marketcaps1)
            marketcaps.append(years1)
            
    else:
        marketcaps = 'NOTFOUND'
                
    
    financialdict = {}
    financialarray = [revenue, grossprofit, netincome, netprofitmargin, changetoliab, changeincash, totalcashfromoperations, cash, totalassets, totalliabilities, currentratio, quickratio, forwardeps, trailingeps, trailingpe, forwardpe, prices, marketcaps]
    finarraydir = ['revenue', 'grossprofit', 'netincome', 'netprofitmargin', 'changetoliab', 'changeincash', 'totalcashfromoperations', 'cash', 'totalassets', 'totalliabilities', 'currentratio', 'quickratio', 'forwardeps', 'trailingeps', 'trailingpe', 'forwardpe', 'prices', 'marketcaps']
    
    for i in range(len(financialarray)):
        if financialarray[i] != 'NOTFOUND':
            financialdict[finarraydir[i]] = financialarray[i]
    
    
    return financialdict


def stockfil(fdict):
    #how to evaluate stock
    #revenue > 0 for 2,3
    #netincome > 0 for 2,3 
    
    arg1 = 'revenue' in fdict
    arg2 = 'netincome' in fdict
    arg3 = 'totalassets' in fdict
    arg4 = 'totalliabilities' in fdict
    arg5 = 'trailingpe' in fdict
    arg6 = 'marketcaps' in fdict
    arg7 = 'totalcashfromoperations' in fdict
    
    if arg1 == True and arg2 == True and arg3 == True and arg4 == True and arg5 == True and arg6 == True and arg7 == True:
        if len(fdict['marketcaps']) == 2:
            if fdict['marketcaps'][1][-1] == "2021":
                if fdict['revenue'][-1] > 0:
                    if (fdict['revenue'][-1])/(fdict['marketcaps'][0][-1]) > 0.2:
                        if fdict['netincome'][-1] > 0:
                            if (fdict['netincome'][-1])/(fdict['marketcaps'][0][-1]) > 0.15:
                                if fdict['totalassets'][-1] > fdict['totalliabilities'][-1]:
                                    if (fdict['totalcashfromoperations'][-1])/(fdict['marketcaps'][0][-1]) > 0.17:
                                        if fdict['trailingpe'] > 0 and fdict['trailingpe'] < 8:
                                            return "GOOD"
                                        else:
                                            return "SSBAD"
                                    else:
                                        return "SSBAD"
                                else:
                                    return "SBAD"
                            else:
                                return "BAD"
                        else:
                            return "BAD"
                        
                    else:
                        return "BAD"
                else:
                    return "BAD"
            else:
                return "Not enough market cap info"
    
        elif len(fdict['marketcaps']) == 4:
            if fdict['revenue'][-1] > 0:
                    revenue = fdict['revenue'][-1]
                    marketcap = fdict['marketcaps'][-1]
                    netincome = fdict['netincome'][-1]
                    if revenue/marketcap > 0.2:
                        if netincome > 0:
                            if netincome/marketcap > 0.15:
                                if fdict['totalassets'][-1] > fdict['totalliabilities'][-1]:
                                    if (fdict['totalcashfromoperations'][-1])/marketcap > 0.17:
                                        if fdict['trailingpe'] > 0 and fdict['trailingpe'] < 8:
                                            return "Good"
                                        else:
                                            return "SSBAD"
                                    else:
                                        return "SSBAD"
                                else:
                                    return "SBAD"
                            else:
                                return "BAD"
                        else:
                            return "BAD"
                        
                    else:
                        return "BAD"
            else:
                return "BAD"
        else:
            return "Not enough market cap info"
            
    elif arg1 == True and arg2 == True and arg5 == True and arg6 == True:
        if fdict['revenue'][-1] > 0:
            if len(fdict['marketcaps']) == 4:
                revenue = fdict['revenue'][-1]
                marketcap = fdict['marketcaps'][-1]
                netincome = fdict['netincome'][-1]
                if revenue/marketcap > 0.2:
                    if netincome/marketcap > 0.15:
                        if fdict['trailingpe'] > 0 and fdict['trailingpe'] < 8:
                            return "Good but limited information"
                        else:
                            return "BAD"
                    else:
                        return "BAD"
                else:
                    return "BAD"
            elif len(fdict['marketcaps']) == 2:
                revenue = fdict['revenue'][-1]
                marketcap = fdict['marketcaps'][0][-1]
                netincome = fdict['netincome'][-1]
                if revenue/marketcap > 0.2:
                    if netincome/marketcap > 0.15:
                        if fdict['trailingpe'] > 0 and fdict['trailingpe'] < 8:
                            return "Good but limited information"
                        else:
                            return "BAD"
                    else:
                        return "BAD"
                else:
                    return "BAD"
                
            else:
                return "Not enough market cap information"
        else:
            return "BAD"
    
    else:
        return "Not much information present"

def get_tickers():
    stocklist = pd.read_csv('nasdaq_screener.csv')
    stocklist = pd.DataFrame(stocklist)
    stocklist = stocklist.reset_index()

    tickers = []

    x = 0
    for i in range(len(stocklist.index)):
        tickers.append(stocklist.loc[i, 'Symbol'])

    return tickers


def make_data(fdict, stringsymbol):
    
    arg = 'trailingpe' in fdict
    arg1 = 'revenue' in fdict
    arg2 = 'netincome' in fdict
    arg3 = 'marketcaps' in fdict
    arg4 = 'prices' in fdict
    arg6 = 'totalassets' in fdict
    arg7 = 'totalliabilities' in fdict
    
    if arg1 == False or arg2 == False or arg3 == False or arg4 == False or arg6 == False or arg7 == False:
        return "NotEnoughData"
    
    global yeardata
    if yeardata != '2022':
        length = 2
        index21 = -1
        index20 = -2
        
    else:
        length = 3
        index21 = -2
        index20 = -3
    
    if len(fdict['revenue']) >= length and len(fdict['netincome']) >= length and len(fdict['marketcaps']) > length and len(fdict['prices']) >= length and len(fdict['totalassets']) >= length and len(fdict['totalliabilities']) >= length:
        revenue = [fdict['revenue'][index20], fdict['revenue'][index21]]
        netincome = [fdict['netincome'][index20], fdict['netincome'][index21]]
        marketcaps = [fdict['marketcaps'][index20], fdict['marketcaps'][index21]]
        prices = [fdict['prices'][-2], fdict['prices'][-1]]
        if arg == True:
            trailingpe = fdict['trailingpe']
        totalassets = [fdict['totalassets'][index20], fdict['totalassets'][index21]]
        totalliabilities = [fdict['totalliabilities'][index20], fdict['totalliabilities'][index21]]
        pchange = (prices[1] - prices[0])/(prices[0])
        
        dat20 = {
            "symbol": [stringsymbol],
            "revenue": [revenue[0]],
            "netincome": [netincome[0]],
            "marketcap": [marketcaps[0]],
            "prices": [prices[0]],
            "totalassets": [totalassets[0]],
            "totalliabilities": [totalliabilities[0]],
            "percentchange": [pchange]
        }
        
        df20 = pd.DataFrame(data=dat20)
        
        global df2020
        df2020 = df2020.append(df20)
        del df20
        del dat20
        
        
        dat21 = {
            "symbol": [stringsymbol],
            "revenue": [revenue[1]],
            "netincome": [netincome[1]],
            "marketcap": [marketcaps[1]],
            "prices": [prices[1]],
            "totalassets": [totalassets[1]],
            "totalliabilities": [totalliabilities[1]],
        }
        
        df21 = pd.DataFrame(data=dat21)
        
        global df2021
        df2021 = df2021.append(df21)
        del df21
        del dat21
        
                
data2020 = {
    "symbol": ['Ahah'],
    "revenue": [0],
    "netincome": [0],
    "marketcap": [0],
    "prices": [0],
    "totalassets": [0],
    "totalliabilities": [0],
    "percentchange": [0]
}

global df2020
df2020 = pd.DataFrame(data= data2020)
    
data2021 = {
    "symbol": ['Ahah'],
    "revenue": [0],
    "netincome": [0],
    "marketcap": [0],
    "prices": [0],
    "totalassets": [0],
    "totalliabilities": [0]
}

global df2021
df2021 = pd.DataFrame(data= data2021)
        
        
        

goodstocks = []
stocks = get_tickers()
#stakcs of 797, 5 times = 3985
for x in range(100):
    arg = x % 20
    if arg == 1:
        time.sleep(10)
        print('timeout')
    yfsymbol = yf.Ticker(stocks[x])
    stringsymbol = stocks[x]
    print(stringsymbol)
    findict = getvalues(yfsymbol, stringsymbol)
    make_data(findict, stringsymbol)

    global yeardata
    yeardata = ''

print(goodstocks)

global df2020
print(df2020)

global df2021
print(df2021)

    
#ACON

        
#apple = yf.Ticker("SND")
#stringsymbol = "SND"
#findict = getvalues(apple, stringsymbol)

#print(stockfil(findict))









AACI
[*********************100%***********************]  1 of 1 completed


/var/folders/dv/3kgy00r121jfkkn0cj9v98f80000gn/T/ipykernel_1234/3426891809.py:68: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/sumandeepchahal/Desktop/StockProgramming/chromedriver')


timeout
AACIW
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AACIW: No data found for this date range, symbol may be delisted
AADI
[*********************100%***********************]  1 of 1 completed
AAL
[*********************100%***********************]  1 of 1 completed
AAME
[*********************100%***********************]  1 of 1 completed
AAOI
[*********************100%***********************]  1 of 1 completed
AAON
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
AATC
[*********************100%***********************]  1 of 1 completed
AAWW
[*********************100%***********************]  1 of 1 completed
ABCB
[*********************100%***********************]  1 of 1 completed
ABEO
[*********************100%***********************]  1 of 1 completed
ABGI
[*********************100%***********************]  1 of 1 completed
ABIO
[***********

[*********************100%***********************]  1 of 1 completed
AEAEW
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AEAEW: No data found for this date range, symbol may be delisted
AEHA
[*********************100%***********************]  1 of 1 completed
AEHAW
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AEHAW: No data found for this date range, symbol may be delisted
AEHR
[*********************100%***********************]  1 of 1 completed
AEI
[*********************100%***********************]  1 of 1 completed
AEIS
[*********************100%***********************]  1 of 1 completed
AEMD
[*********************100%***********************]  1 of 1 completed
AEP
[*********************100%***********************]  1 of 1 completed
AEPPZ
[*********************100%***********************]  1 of 1 completed
AERC
[*********************100%***********************]  1 of 1 completed
AERI
[**********

In [3]:
global df2020
df2020.to_csv('2020.csv')


In [4]:
global df2021
df2021.to_csv('2021.csv')